In [2]:
# !pip install routingpy

In [3]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from routingpy.routers import MapboxOSRM
import numpy as np

In [4]:
key = 'pk.eyJ1Ijoic2hhdW5ob2FuZyIsImEiOiJjbHJmczAyeXcwN2h5MmpydjZxMnAwbG1lIn0.IEy0cipw3KOQChIQa2hZxQ'

In [5]:
mb = MapboxOSRM(api_key = key)

In [54]:
# read bus stops geojson
bus_stops = gpd.read_file('data/Bus_Stops.geojson')
bus_stops = bus_stops.drop(columns = ['DATE_UPDATED'])
bus_stops.shape

(21346, 13)

In [21]:
def mb_isochrone(gdf, time = [5, 10, 15], profile = "walking"):

    # Grab X and Y values in 4326
    gdf['LON_VALUE'] = gdf.to_crs(4326).geometry.x
    gdf['LAT_VALUE'] = gdf.to_crs(4326).geometry.y

    coordinates = gdf[['LON_VALUE', 'LAT_VALUE']].values.tolist()

    # Build a list of shapes
    isochrone_shapes = []

    if type(time) is not list:
        time = [time]

    # Use minutes as input, but the API requires seconds
    time_seconds = [60 * x for x in time]

    # iterate through the list of coordinate pairs, then iterate through the object returned and extract the isochrone geometries.  
    for c in coordinates:
        iso_request = mb.isochrones(locations = c, profile = profile,
                                    intervals = time_seconds, polygons = "true")

        for i in iso_request:
            iso_geom = Polygon(i.geometry[0])
            isochrone_shapes.append(iso_geom)

    # Here, we re-build the dataset but with isochrone geometries
    df_values = gdf.drop(columns = ['geometry', 'LON_VALUE', 'LAT_VALUE'])

    time_col = time * len(df_values)

    # We'll need to repeat the dataframe to account for multiple time intervals
    df_values_rep = pd.DataFrame(np.repeat(df_values.values, len(time_seconds), axis = 0))
    df_values_rep.columns = df_values.columns

    isochrone_gdf = gpd.GeoDataFrame(
        data = df_values_rep,
        geometry = isochrone_shapes,
        crs = 4326
    )

    isochrone_gdf['time'] = time_col

    # We are sorting the dataframe in descending order of time to improve visualization
    isochrone_gdf = isochrone_gdf.sort_values('time', ascending = False)

    return(isochrone_gdf)

In [ ]:
# split the bus stops dataframe to a dataframe of 100 rows each
for i in range(0, len(bus_stops), 500):
    sample = bus_stops.iloc[i:i+500]
    sample = mb_isochrone(sample, time = [10], profile = "walking")
    sample.to_file(f'isochrones_{i}.geojson', driver = 'GeoJSON')

In [74]:
# go through the folder and read all the files with *.geojson
import os
import glob

files = glob.glob('*.geojson')
# read all the files and append them to a single dataframe
isochrones = pd.DataFrame()
isochrones = gpd.read_file(files[0]) #import column headers from the first file

for f in files:
    df = gpd.read_file(f)
    isochrones = pd.concat([isochrones, df[1:]])

isochrones.shape

(999, 14)

In [75]:
isochrones.explore()